In [39]:
# ruff: noqa: F401, E402
import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from multiprocess import Process

import arrays
import combinatorics
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    combinatorics,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from binary_index_tree import BinIndexTree
from combinatorics import fib
from parsing import format_binop_expression, parse_binop_expression
from sequences import find_if
from strings import StringView, splint

print()


In [197]:
from trees import TreeNode, Node


def inorder_postorder_tangle(ino: list, post: list) -> Node:
    def r(i: int, j: int):
        if i < j:
            n = TreeNode(next(it))
            n.right, n.left = r(idx[n.data] + 1, j), r(i, idx[n.data])
            return n
        return None

    idx = [0] * (max(ino) + 1)
    for i, v in enumerate(ino):
        idx[v] = i
    it = reversed(post)
    return r(0, len(ino))


In [200]:
r = inorder_postorder_tangle([4, 8, 2, 5, 1, 6, 3, 7], [8, 4, 5, 2, 6, 7, 3, 1])
r.display()
print(r.serialize("in"))


   _1_  
  /   \ 
 _2   3 
/  \ / \
4  5 6 7
 \      
 8      
(((() 4 8) 2 5) 1 (6 3 7))


In [164]:
t = trees.make("1 2 3")
t.display()
print(list(t.inorder()))
print(list(t.postorder()))


 1 
/ \
2 3
[2, 1, 3]
[2, 3, 1]


In [165]:
t = trees.make("2 N 1 N 3")
t.display()
print(list(t.inorder()))
print(list(t.postorder()))


2  
 \ 
 1 
  \
  3
[2, 1, 3]
[3, 1, 2]


In [166]:
t = trees.make("2 N 3 1")
t.display()
print(list(t.inorder()))
print(list(t.postorder()))


2_ 
  \
  3
 / 
 1 
[2, 1, 3]
[1, 3, 2]


In [169]:
t = trees.make("1 N 3 N 2")
t.display()
print(list(t.inorder()))
print(list(t.postorder()))


1  
 \ 
 3 
  \
  2
[1, 3, 2]
[2, 3, 1]


In [170]:
t = trees.make("1 N 3 2")
t.display()
print(list(t.inorder()))
print(list(t.postorder()))


1_ 
  \
  3
 / 
 2 
[1, 2, 3]
[2, 3, 1]
